In [1]:
import pandas as pd

In [3]:
v1_res = pd.read_csv('results/BTC_trades.csv')
v2_res = pd.read_csv('results/BTC_trades_backtestingpy.csv')


In [5]:
v1_res.pnl.sum()

np.float64(315044.9226230075)

In [10]:
v2_res.PnL.sum()


np.float64(307829.89999999997)

In [5]:
import akshare as ak

fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="513100", 
                                          period="daily", 
                                          start_date="20200101", end_date="20250512", adjust="")
print(fund_etf_hist_em_df)

              日期     开盘     收盘     最高     最低      成交量          成交额    振幅  \
0     2020-01-02  3.201  3.240  3.243  3.201   160678   51681636.0  1.32   
1     2020-01-03  3.270  3.254  3.270  3.233   232830   75798968.0  1.14   
2     2020-01-06  3.228  3.212  3.232  3.196   188451   60602918.0  1.11   
3     2020-01-07  3.231  3.257  3.280  3.231   187465   60875890.0  1.53   
4     2020-01-08  3.226  3.219  3.243  3.194   204366   65796355.0  1.50   
...          ...    ...    ...    ...    ...      ...          ...   ...   
1289  2025-05-06  1.469  1.457  1.470  1.454  6441272  939843165.0  1.11   
1290  2025-05-07  1.463  1.462  1.476  1.459  3423798  501356837.0  1.17   
1291  2025-05-08  1.477  1.485  1.486  1.470  4591589  678513366.0  1.09   
1292  2025-05-09  1.485  1.486  1.489  1.479  5095210  756114122.0  0.67   
1293  2025-05-12  1.513  1.516  1.518  1.506  5584639  844555802.0  0.81   

       涨跌幅    涨跌额   换手率  
0     1.47  0.047  0.17  
1     0.43  0.014  0.25  
2    -1.2